<a href="https://colab.research.google.com/github/wendy705/guen_test_message/blob/main/Test_TextCortex_Guendalina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 0. Installing frameworks and libraries

(I never put it on quiet because sometimes the Colab Runtimes need to be restarted if there is a pip conflict)

In [2]:
!pip install pytorch_lightning
!pip install datasets

     |████████████████████████████████| 527 kB 16.6 MB/s 
     |████████████████████████████████| 829 kB 55.7 MB/s 
     |████████████████████████████████| 952 kB 37.1 MB/s 
     |████████████████████████████████| 134 kB 70.7 MB/s 
     |████████████████████████████████| 596 kB 45.0 MB/s 
     |████████████████████████████████| 397 kB 72.0 MB/s 
     |████████████████████████████████| 1.1 MB 47.2 MB/s 
     |████████████████████████████████| 271 kB 67.6 MB/s 
     |████████████████████████████████| 144 kB 74.7 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=a921d663758ca0ff943655bc98ed452cbd049b4831e28bd1730b627fc91b369b
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfu

     |████████████████████████████████| 325 kB 12.7 MB/s 
     |████████████████████████████████| 212 kB 70.1 MB/s 
     |████████████████████████████████| 67 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 75.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
# installing necessary libraries
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 1.2 MB 14.9 MB/s 
     |████████████████████████████████| 3.8 MB 9.6 MB/s 
     |████████████████████████████████| 6.5 MB 53.5 MB/s 
     |████████████████████████████████| 895 kB 50.6 MB/s 


In [4]:
# Let's check the GPU we have
!nvidia-smi

Wed Mar 16 21:59:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 1. Selecting a Dataset

I don't remember where, I read something that really stuck with me. "In Data Science Data comes before the Science". Data is the key to a successful model. Unfortunately, data is at the same time abundant and scarce. While we have an enormous amount of data available, sometimes we can't find exactly what we want. So we need to get a bit creative.

The aim of this exercise, if I understood correctly, is to give the machine a prompt. e.g. the key points of the message we want to send/the text we want to write, and have the model write the rest for us. Thinking about it, it is almost as if we wanted to write the subject of our email, and then have the email write itself - if only! 

So I decided to use the aeslc dataset. The dataset is composed of email and subject line pairs. Normally, the email is given as the input, and we want the model to extract the subject line. In this case, we are switching things up. 

One of the advantages of this dataset is that it is already available in the HuggingFace datasets library. So it is already clean, nicely formatted, and ready to use. Let's have a look to see what we are dealing with.

In [4]:
from datasets import load_dataset

raw_dataset_aeslc = load_dataset("aeslc", 'default')
raw_dataset_aeslc

Using custom data configuration default


Generating train split:   0%|          | 0/14436 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1960 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1906 [00:00<?, ? examples/s]

Dataset aeslc downloaded and prepared to /root/.cache/huggingface/datasets/aeslc/default/1.0.0/eb8e30234cf984a58ebe9f205674597ac1db2ec91e7321cd7f36864f7e3671b8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 14436
    })
    validation: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1960
    })
    test: Dataset({
        features: ['email_body', 'subject_line'],
        num_rows: 1906
    })
})

###2. Select a model from the HuggingFace Transformers library and Load the Data

Now that we have our input and output data, we are going to select a model from the HuggingFace transformer library. This allows us to use a pre-trained model, that has been trained on a large amount of data to perform several different tasks. This is what we call the body of a model. We are then going to fine-tune it for a specific task, meaning we are going to create the head of our model.

For this test, we are going to use the [Google T5 model](https://huggingface.co/t5-base). You can use its multilingual version, MT5, to train your model in another language. The MT5 model supports 101 languages. The T5-base model has 220 million parameters, which means it is quite light compared to other models, but at the same time it is flexible and powerful enough to learn new tasks with simple fine-tuning ([Google AI blog](https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html)).

### 3. Fine-Tuning the Model to create a new Head

Here we define the model and start setting everything up for our fine-tuning process.

As you can see, there are a few steps we need to take in order to make sure we have all the pieces necessary to train our transformer.

In [5]:
# importing a bunch of libraries and tools we will need
import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
# we are going to set a random seed to make sure our experiments can be easily reproduced
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)

set_seed(42)

In [11]:
# we are going to define a T5FineTuner class that extends the LightningModule by PyTorch
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams

        self.model = T5ForConditionalGeneration.from_pretrained(hparams['model_name_or_path'])
        self.tokenizer = T5Tokenizer.from_pretrained(hparams['tokenizer_name_or_path'])

    def hparams(self):
        return self.hparams

    def is_logger(self):
        return True #self.trainer.proc_rank <= 0

    def forward(
            self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        # return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams['weight_decay'],
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams['learning_rate'], eps=self.hparams['adam_epsilon'])
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch=None, batch_idx=None, optimizer=None, optimizer_idx=None, optimizer_closure=None,  second_order_closure=None, on_tpu=False, using_native_amp=False, using_lbfgs=False):
        # if self.trainer.use_tpu:
        #     xm.optimizer_step(optimizer)
        # else:
        optimizer.step(closure=optimizer_closure)
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams['train_batch_size'], drop_last=True, shuffle=True,
                                num_workers=4)
        t_total = (
                (len(dataloader.dataset) // (self.hparams['train_batch_size'] * max(1, self.hparams['n_gpu'])))
                // self.hparams['gradient_accumulation_steps']
                * float(self.hparams['num_train_epochs'])
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams['warmup_steps'], num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="validation", args=self.hparams)
        return DataLoader(val_dataset, batch_size=self.hparams['eval_batch_size'], num_workers=4)

logger = logging.getLogger(__name__)

# a little callback function to log and save the results of our training
class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.hparams["output_dir"], "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

Perfect, now we just have to set the arguments of our model

In [8]:
# let's set the arguments to train our models
args_dict = dict(
    data_dir="data", # path for data files - not necessary in our case since the data is available in the datasets library
    output_dir="textcortext_test_aeslc", # path to save the checkpoints
    model_name_or_path='t5-base',
    tokenizer_name_or_path='t5-base',
    max_seq_length=256,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=5,
    gradient_accumulation_steps=16,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
)

tokenizer = T5Tokenizer.from_pretrained('t5-base')

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

We can now create a class that extends Pytorch's Dataset, in order to correctly process our data and build our input and target tensors

In [9]:
class MessageDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, max_len=512):

        self.source_column = "subject_line"
        self.target_column = "email_body"
        self.raw_data = load_dataset("aeslc", 'default')
        self.data = pd.DataFrame(self.raw_data[type_path])

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()
        target_mask = self.targets[index]["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            input_, target = self.data.loc[idx, self.source_column], self.data.loc[idx, self.target_column]

            input_ = "elaborate: "+ input_ + ' </s>'
            target = target + " </s>"

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt", truncation='longest_first'
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt", truncation='longest_first'
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

We can finally start training. In the first lines we are going to print a few pieces of information that can help us better visualise the process.


In [12]:
dataset = MessageDataset(tokenizer, 'data', 'validation', 256)
print("Val dataset: ",len(dataset))

data = dataset[61]
print(tokenizer.decode(data['source_ids']))
print(tokenizer.decode(data['target_ids']))

# checking if the path to save the files exist, otherwise we create it on the fly
if not os.path.exists('textcortext_test_aeslc'):
    os.makedirs('textcortext_test_aeslc')

# setting the arguments and printing them
args = args_dict
print(args)

# checkpoint callback function, we monitor the validation loss and save top_k=5
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args['output_dir'], monitor="val_loss", mode="min", save_top_k=5
)

train_params = dict(
    accumulate_grad_batches=args['gradient_accumulation_steps'],
    gpus=args['n_gpu'],
    max_epochs=args['num_train_epochs'],
 #   early_stop_callback=False,
    precision= 16 if args['fp_16'] else 32,
    amp_level=args['opt_level'],
    amp_backend='apex',
    gradient_clip_val=args['max_grad_norm'],
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

def get_dataset(tokenizer, type_path, args):
  return MessageDataset(tokenizer=tokenizer, data_dir=args["data_dir"], type_path=type_path,  max_len=args['max_seq_length'])

print ("Initialize model")
model = T5FineTuner(args)

trainer = pl.Trainer(**train_params)

print (" Training model")
trainer.fit(model)

print ("training finished")

print ("Saving model")
model.model.save_pretrained('textcortext_test_aeslc')

print ("Model saved")

!cp "/content/textcortext_test_aeslc/" -a "/content/drive/My Drive/"
!cp "/content/lightning_logs/" -a "/content/drive/My Drive/"
print ("Copied the final folder to Google Drive")

Using custom data configuration default
Reusing dataset aeslc (/root/.cache/huggingface/datasets/aeslc/default/1.0.0/eb8e30234cf984a58ebe9f205674597ac1db2ec91e7321cd7f36864f7e3671b8)


  0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Val dataset:  1960
elaborate: Announcement for all EGM employees</s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

training finished
Saving model
Model saved
Copied the final folder to Google Drive


## 4. Inference
Now that we have trained and saved the model, we can start inference.

In [5]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

def set_seed(seed):
  torch.manual_seed(seed)
#  if torch.cuda.is_available():
#   torch.cuda.manual_seed_all(seed)

set_seed(42)

best_model_path = "/content/drive/MyDrive/textcortext_test_aeslc"
model = T5ForConditionalGeneration.from_pretrained('Guen/guen_test_prompt_generation')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

device  cuda


In [6]:
sentence_1 = "follow up on last thursday's meeting regarding new software feature"
sentence_2 = "I'll be out of office from 15th to the 25th. meeting on thursday 13th"
sentence_3 = "client not happy with product. need to call meeting. production problem"
sentence_4 = "holiday party at the office on friday"
sentence_5 = "from head of department to marketing team: need update on latest campaign"
sentence_6 = "IT issue - our server is offline"
sentence_7 = "linda is going on maternity leave. we are going to send her a present from the entire department"
sentence_8 = "update on product delivery deadline"
sentence_9 = "inquiry about prices and packages from new prospect."
sentence_10 = "IT system update on friday 13th. save your work and log off"
sentence = sentence_1

text =  "elaborate: " + sentence + " </s>"

max_len = 350

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=20
)

print ("\nPrompt: ")
print (sentence)
print ("\n")
print ("Message: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:195: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."



Original sentence: 
I'll be out of office from 15th to the 25th. meeting on thursday 13th


Paraphrased sentences: 
0: Bez, Mike will be out of the office Thursday, 13th..from 9:00 am to 1:00 am. I think I have all of my business done and I will be back in the office mid-afternoon, not long after that. Your help is greatly appreciated. I'll be back in the office on Friday at 7:44 pm. I will be able to help in case you need anything. Thanks,
1: I will be out of the office from thursday, 13th. This month I'll be back on the desk - if your haven't heard from me already, so let's discuss. Best Regards
2: We will be coming in on September 13th on my final visit back in Houston, but then back in Houston on Monday for a meeting, which starts at 6:00 p.m. I'm expecting I can be here about 2:00 p.m., but if you have any questions, please let me know. Thank you for your assistance in this matter. I am on vacation all week, because everyone here sees me and love your company. And I hope that the

Now, since we are happy with our results, let's push our model to the HuggingFace model hub!

In [7]:
# login to huggingface
!pip install huggingface_hub

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [18]:
# we are going to save it in pytorch
pt_model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/guen_test_prompt_generation")
pt_model.save_pretrained("/content/drive/MyDrive/guen_test_prompt_generation")

In [19]:
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


In [20]:
pt_model.push_to_hub("guen_test_prompt_generation")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/Guen/guen_test_prompt_generation into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.34k/850M [00:00<?, ?B/s]

remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified
remote: tput: No value for $TERM and no -T specified        
remote: tput: No value for $TERM and no -T specified        
To https://huggingface.co/Guen/guen_test_prompt_generation
   61e72f3..11ff0e7  main -> main



'https://huggingface.co/Guen/guen_test_prompt_generation/commit/11ff0e70529609b26b2ed7f04c2833298b4d49fa'